## Avance 2 - Proyecto Final

In [6]:
from src.db.database import DBConnection

In [7]:
## Verificar Patron Singleton
db1 = DBConnection()
db2 = DBConnection()

print("Es la misma instancia?", db1 is db2)
print("Es la misma base de datos?", db1.engine.url == db2.engine.url)

Es la misma instancia? True
Es la misma base de datos? True


In [8]:
# Ejecutar una query con df
df = db1.execute_query("SELECT * FROM customers LIMIT 5")
df

2025-06-02 00:03:31,488 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-02 00:03:31,489 INFO sqlalchemy.engine.Engine SELECT * FROM customers LIMIT 5
2025-06-02 00:03:31,489 INFO sqlalchemy.engine.Engine [cached since 61.93s ago] {}
2025-06-02 00:03:31,491 INFO sqlalchemy.engine.Engine ROLLBACK


,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address
0,1,Stefanie,Y,Frye,79,97 Oak Avenue
1,2,Sandy,T,Kirby,96,52 White First Freeway
2,3,Lee,T,Zhang,55,921 White Fabien Avenue
3,4,Regina,S,Avery,40,75 Old Avenue
4,5,Daniel,S,Mccann,2,283 South Green Hague Avenue
